In [ ]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
!pip install --no-deps "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-7B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, 
    bias = "none",    
    use_gradient_checkpointing = "unsloth", 
    random_state = 3407,
)

In [ ]:
import json
from datasets import Dataset

prompt_style = """Отредактируй описание товара для Авито. 
Будь грамотным, добавь в текст структуру и привлекательность, а также строго придерживайся фактов из исходного текста.

### Категория:
{}

### Исходное описание:
{}

### Улучшенное описание:
{}"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    categories = examples["base_category_name"] + " / " + examples["base_subcategory_name"]
    originals  = examples["base_description"]
    generateds = examples["generated_description"]
    texts = []
    for cat, orig, gen in zip(categories, originals, generateds):
        text = prompt_style.format(cat, orig, gen) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

data = []
dataset_path = "/kaggle/input/твоя-папка/avito_train_dataset.jsonl" 

with open(dataset_path, "r", encoding="utf-8") as f:
    for line in f:
        data.append(json.loads(line))

full_dataset = Dataset.from_list(data)

dataset_split = full_dataset.train_test_split(test_size=0.05, seed=42)
train_dataset = dataset_split["train"]
test_dataset  = dataset_split["test"]

train_dataset = train_dataset.map(formatting_prompts_func, batched = True,)

print(f"Размер обучающей выборки: {len(train_dataset)}")
print(f"Размер тестовой выборки: {len(test_dataset)}")

test_dataset.to_json("test_samples_holdout.jsonl", force_ascii=False)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

trainer_stats = trainer.train()